In [1]:
from dotenv import load_dotenv
import os
load_dotenv()

True

In [2]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

In [3]:
import pandas as pd
from src.config import PROCESSED_DATA_DIR

ts_data = pd.read_parquet(PROCESSED_DATA_DIR / "ts_data.parquet")
ts_data

,start_hour,start_station_id,rides
0,2023-01-01 00:00:00,5905.140137,0
1,2023-01-01 01:00:00,5905.140137,5
2,2023-01-01 02:00:00,5905.140137,7
3,2023-01-01 03:00:00,5905.140137,3
4,2023-01-01 04:00:00,5905.140137,2
...,...,...,...
59107,2025-03-31 19:00:00,6822.089844,23
59108,2025-03-31 20:00:00,6822.089844,14
59109,2025-03-31 21:00:00,6822.089844,5
59110,2025-03-31 22:00:00,6822.089844,2


In [4]:
import pytz

est = pytz.timezone("US/Eastern")
ts_data["start_hour"] = ts_data["start_hour"].dt.tz_localize(est, ambiguous="NaT", nonexistent="NaT")
ts_data = ts_data.dropna(subset=["start_hour"])
ts_data

,start_hour,start_station_id,rides
0,2023-01-01 00:00:00-05:00,5905.140137,0
1,2023-01-01 01:00:00-05:00,5905.140137,5
2,2023-01-01 02:00:00-05:00,5905.140137,7
3,2023-01-01 03:00:00-05:00,5905.140137,3
4,2023-01-01 04:00:00-05:00,5905.140137,2
...,...,...,...
59107,2025-03-31 19:00:00-04:00,6822.089844,23
59108,2025-03-31 20:00:00-04:00,6822.089844,14
59109,2025-03-31 21:00:00-04:00,6822.089844,5
59110,2025-03-31 22:00:00-04:00,6822.089844,2


In [5]:
import hopsworks

api_key = os.getenv('HOPSWORKS_API_KEY')
project_name = os.getenv('HOPSWORKS_PROJECT_NAME')

# pip install confluent-kafka
# Initialize connection to Hopsworks
project = hopsworks.login(
    api_key_value=api_key,
    project=project_name
)
print(f"Successfully connected to Hopsworks project: {project_name}")

2025-05-04 15:43:36,503 INFO: Initializing external client
2025-05-04 15:43:36,505 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-05-04 15:43:37,541 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1212635
Successfully connected to Hopsworks project: nolanphi


In [15]:
feature_store = project.get_feature_store()

In [16]:
FEATURE_GROUP_NAME = "bike_time_series_hourly_feature_group"
FEATURE_GROUP_VERSION = 1

In [17]:
feature_group = feature_store.get_or_create_feature_group(
    name=FEATURE_GROUP_NAME,
    version=FEATURE_GROUP_VERSION,
    description="Time-series data at hourly frequency",
    primary_key=["start_station_id", "start_hour"],
    event_time="start_hour"
)

In [18]:
feature_group.insert(ts_data, write_options={"wait_for_job": False})

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1212635/fs/1200268/fg/1440415


Uploading Dataframe: 100.00% |██████████| Rows 59097/59097 | Elapsed Time: 00:04 | Remaining Time: 00:00


Launching job: bike_time_series_hourly_feature_group_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1212635/jobs/named/bike_time_series_hourly_feature_group_1_offline_fg_materialization/executions


(Job('bike_time_series_hourly_feature_group_1_offline_fg_materialization', 'SPARK'),
 None)

In [13]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 59097 entries, 0 to 59111
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype                     
---  ------            --------------  -----                     
 0   start_hour        59097 non-null  datetime64[ns, US/Eastern]
 1   start_station_id  59097 non-null  float32                   
 2   rides             59097 non-null  int16                     
dtypes: datetime64[ns, US/Eastern](1), float32(1), int16(1)
memory usage: 1.2 MB
